In [2]:
import time
import os
import cv2
import numpy as np
import pandas as pd
import subprocess
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from PIL import Image

In [9]:
def record_videos(duration_s=5, fps=30, resolution=(640, 480)):
    folder_path = f"../data/video/"
    os.makedirs(folder_path, exist_ok=True)
    
    formats = {
        "i420.avi": cv2.VideoWriter_fourcc(*"I420"),
        "mp4.mp4": cv2.VideoWriter_fourcc(*"mp4v"),
        "divx.avi": cv2.VideoWriter_fourcc(*"DIVX"),
        "flv.flv": cv2.VideoWriter_fourcc(*"FLV1"),
        "xvid.avi": cv2.VideoWriter_fourcc(*"XVID")
    }
    
    cap = cv2.VideoCapture(0)
    writers = {fmt: cv2.VideoWriter(os.path.join(folder_path, fmt), fourcc, fps, resolution) for fmt, fourcc in formats.items()}
    
    frame_count = fps * duration_s
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        for writer in writers.values():
            writer.write(frame)
    
    cap.release()
    for writer in writers.values():
        writer.release()

def extract_frames(folder_name):
    folder_path = f"../data/video/"
    output_path = os.path.join(folder_path, "frames")
    os.makedirs(output_path, exist_ok=True)
    
    frame_files = {}
    for video_file in os.listdir(folder_path):
        if not video_file.endswith((".avi", ".mp4", ".flv")):
            continue
        
        cap = cv2.VideoCapture(os.path.join(folder_path, video_file))
        ret, frame = cap.read()
        if ret:
            frame_filename = os.path.join(output_path, f"frame_{video_file.split('.')[0]}.jpg")
            cv2.imwrite(frame_filename, frame)
            frame_files[video_file] = frame_filename
        cap.release()
    
    return frame_files

def analyze_videos(folder_name):
    folder_path = f"../data/video/"
    frame_files = extract_frames(folder_name)
    
    data = []
    for video_file, frame_file in frame_files.items():
        file_path = os.path.join(folder_path, video_file)
        file_size = os.path.getsize(file_path) / 1024  # KB
        
        img = cv2.imread(frame_file, cv2.IMREAD_GRAYSCALE)
        ref_img = cv2.imread(list(frame_files.values())[0], cv2.IMREAD_GRAYSCALE)
        
        ssim_score = ssim(ref_img, img) if img is not None else None
        mse_score = np.mean((ref_img.astype("float") - img.astype("float")) ** 2) if img is not None else None
        
        height, width = img.shape[:2]
        resolution = f"{width}x{height}"
        
        duration_s = 5
        fps = 5
        data.append([folder_name, video_file.split('.')[1].upper(), duration_s, fps, resolution, f"{file_size:.2f} KB", round(ssim_score, 4), round(mse_score, 4)])
    
    df = pd.DataFrame(data, columns=["Test No", "Format", "Duration (s)", "FPS", "Resolution", "File Size", "SSIM", "MSE"])
    return df


In [10]:
# Run processes
test_folder = "test_video"
record_videos(test_folder)
df = analyze_videos(test_folder)

# Plot file size vs format
plt.figure(figsize=(8, 5))
plt.bar(df["Format"], df["File Size"].str.replace(" KB", "").astype(float), color=["red", "blue", "green", "purple", "orange"])
plt.xlabel("Video Format")
plt.ylabel("File Size (KB)")
plt.title("File Size vs. Video Format")
plt.grid(axis="y")
plt.show()

# Plot image quality vs format
plt.figure(figsize=(8, 5))
plt.bar(df["Format"], df["SSIM"].astype(float), color=["red", "blue", "green", "purple", "orange"])
plt.xlabel("Video Format")
plt.ylabel("SSIM Score")
plt.title("Image Quality vs. Video Format")
plt.grid(axis="y")
plt.show()

df.head()

OpenCV: FFMPEG: tag 0x31564c46/'FLV1' is not supported with codec id 21 and format 'flv / FLV (Flash Video)'
OpenCV: FFMPEG: fallback to use tag 0x00000002/'????'


TypeError: 'str' object cannot be interpreted as an integer